Setup (paths, imports, seed)

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd

from src.config import get_paths, set_seed
from src.io import read_table, save_table
from src.modeling import (
    eval_stratifiedkfold_multiclass_lr,
    eval_stratifiedkfold_multiclass_lgbm,
    eval_kfold_regression_elasticnet,
    eval_kfold_regression_lgbm,
)

paths = get_paths()
set_seed(42)

LABELS_ORDER = ["GC", "G2", "G5", "G7", "G14"]

Load processed datasets

In [2]:
X = pd.read_csv(paths.processed / "X_radiomics.csv")
y_group = pd.read_csv(paths.processed / "y_group.csv")["group"]
y_days = pd.read_csv(paths.processed / "y_days.csv")["days_post_btx"]

# Safety checks
assert len(X) == len(y_group) == len(y_days), "X/y size mismatch"
assert X.shape[1] > 0, "X has no features"

print("Loaded datasets:")
print(f"- X_radiomics: {X.shape}")
print(f"- y_group:     {y_group.shape} | unique={sorted(y_group.unique().tolist())}")
print(f"- y_days:      {y_days.shape}  | min={y_days.min()} max={y_days.max()}")

Loaded datasets:
- X_radiomics: (571, 105)
- y_group:     (571,) | unique=['G14', 'G2', 'G5', 'G7', 'GC']
- y_days:      (571,)  | min=0 max=14


ERAT classification: Logistic Regression (multinomial)

In [3]:
df_folds_lr, summary_lr, cm_lr, oof_lr = eval_stratifiedkfold_multiclass_lr(
    X=X,
    y=y_group,
    labels_order=LABELS_ORDER,
    n_splits=5,
    seed=42,
)

display(df_folds_lr)
display(summary_lr)
display(cm_lr)
oof_lr.head()

,fold,n_test,balanced_accuracy,weighted_kappa_quadratic,mae_days
0,1,115,0.745606,0.825624,1.173913
1,2,114,0.802273,0.909654,0.675439
2,3,114,0.833333,0.852771,0.526316
3,4,114,0.831818,0.879140,0.596491
4,5,114,0.823485,0.811837,0.631579


,metric,mean,std
0,balanced_accuracy,0.807303,0.036648
1,weighted_kappa_quadratic,0.855805,0.039678
2,mae_days,0.720748,0.259136


,GC,G2,G5,G7,G14
GC,103,8,3,6,0
G2,7,80,17,6,0
G5,9,3,104,5,0
G7,9,15,21,65,0
G14,0,0,0,0,110


,y_true,y_pred
0,GC,GC
1,GC,GC
2,GC,GC
3,GC,GC
4,GC,GC


In [4]:
save_table(oof_lr, paths.results / "tables" / "erat_cls_lr_oof_predictions.csv")

WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_cls_lr_oof_predictions.csv')

ERAT classification: LightGBM

In [5]:
df_folds_lgbm, summary_lgbm, cm_lgbm, oof_lgbm = eval_stratifiedkfold_multiclass_lgbm(
    X=X,
    y=y_group,
    labels_order=LABELS_ORDER,
    n_splits=5,
    seed=42,
    n_estimators=500,
)

display(df_folds_lgbm)
display(summary_lgbm)
display(cm_lgbm)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15349
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 104
[LightGBM] [Info] Start training from score -1.645156
[LightGBM] [Info] Start training from score -1.645156
[LightGBM] [Info] Start training from score -1.558145
[LightGBM] [Info] Start training from score -1.645156
[LightGBM] [Info] Start training from score -1.558145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000833 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15441
[LightGBM] [Info] Number of data points in the train set: 457, number of used features: 104
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.549972
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.560335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000870 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15440
[LightGBM] [Info] Number of data points in the train set: 457, number of used features: 104
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.549972
[LightGBM] [Info] Start training from score -1.647347
[LightGBM] [Info] Start training from score -1.560335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,fold,n_test,balanced_accuracy,weighted_kappa_quadratic,mae_days
0,1,115,0.931061,0.943629,0.391304
1,2,114,0.921212,0.973560,0.263158
2,3,114,0.931061,0.933649,0.289474
3,4,114,0.981818,0.989003,0.061404
4,5,114,0.974242,0.986945,0.078947


,metric,mean,std
0,balanced_accuracy,0.947879,0.027945
1,weighted_kappa_quadratic,0.965357,0.025347
2,mae_days,0.216857,0.142332


,GC,G2,G5,G7,G14
GC,111,2,2,5,0
G2,1,106,2,1,0
G5,1,3,114,3,0
G7,2,4,4,100,0
G14,0,0,0,0,110


In [6]:
save_table(oof_lgbm, paths.results / "tables" / "erat_cls_lgbm_oof_predictions.csv")

WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_cls_lgbm_oof_predictions.csv')

ERAT regression: ElasticNet

In [8]:
df_folds_enet, summary_enet, oof_enet = eval_kfold_regression_elasticnet(
    X=X,
    y=y_days,
    n_splits=5,
    seed=42,
    alpha=0.001,
    l1_ratio=0.5,
)

display(df_folds_enet)
display(summary_enet)
display(oof_enet.head())

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.695e+02, tolerance: 1.022e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.832e+02, tolerance: 1.073e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,fold,n_test,mae,rmse,r2
0,1,115,1.516832,1.863444,0.865880
1,2,114,1.347022,1.727667,0.860903
2,3,114,1.254050,1.634894,0.887252
3,4,114,1.417038,1.758819,0.853059
4,5,114,1.125253,1.441276,0.910477


,metric,mean,std
0,mae,1.332039,0.150366
1,rmse,1.685220,0.158921
2,r2,0.875514,0.023295


,y_true,y_pred
0,0,1.685353
1,0,0.359617
2,0,1.019331
3,0,1.302576
4,0,-0.207391


In [9]:
save_table(df_folds_enet, paths.results / "tables" / "erat_reg_enet_folds.csv")
save_table(summary_enet,  paths.results / "tables" / "erat_reg_enet_summary.csv")
save_table(oof_enet,      paths.results / "tables" / "erat_reg_enet_oof_predictions.csv")

WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_reg_enet_oof_predictions.csv')

ERAT regression: LightGBM

In [10]:
df_folds_lgbm_reg, summary_lgbm_reg, oof_lgbm_reg = eval_kfold_regression_lgbm(
    X=X,
    y=y_days,
    n_splits=5,
    seed=42,
    n_estimators=1000,
)

display(df_folds_lgbm_reg)
display(summary_lgbm_reg)
display(oof_lgbm_reg.head())

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002592 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15336
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 104
[LightGBM] [Info] Start training from score 5.515351
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\modre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


,fold,n_test,mae,rmse,r2
0,1,115,0.717350,1.204503,0.943963
1,2,114,0.709720,1.114855,0.942079
2,3,114,0.575833,1.002878,0.957575
3,4,114,0.732269,1.173566,0.934579
4,5,114,0.606007,0.939307,0.961976


,metric,mean,std
0,mae,0.668236,0.071840
1,rmse,1.087022,0.112878
2,r2,0.948034,0.011385


,y_true,y_pred
0,0,-0.110488
1,0,-0.021282
2,0,-0.073871
3,0,0.118741
4,0,-0.276701


In [11]:
save_table(df_folds_lgbm_reg, paths.results / "tables" / "erat_reg_lgbm_folds.csv")
save_table(summary_lgbm_reg,  paths.results / "tables" / "erat_reg_lgbm_summary.csv")
save_table(oof_lgbm_reg,      paths.results / "tables" / "erat_reg_lgbm_oof_predictions.csv")

WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_reg_lgbm_oof_predictions.csv')

Build ERAT outputs for downstream ERAT×EHAT comparison

In [12]:
erat_days_pred = oof_lgbm_reg["y_pred"].to_numpy(dtype=float)

mn, mx = np.nanmin(erat_days_pred), np.nanmax(erat_days_pred)
erat_norm = np.zeros_like(erat_days_pred) if mx == mn else (erat_days_pred - mn) / (mx - mn)

df_erat_outputs = pd.DataFrame({
    "days_true": y_days.to_numpy(dtype=float),
    "erat_days_pred": erat_days_pred,
    "erat_norm": erat_norm,
    "group": y_group.to_numpy(),
})

display(df_erat_outputs.head())

# Save for notebook 05
save_table(df_erat_outputs, paths.processed / "erat_outputs.csv")

,days_true,erat_days_pred,erat_norm,group
0,0.0,-0.110488,0.029865,GC
1,0.0,-0.021282,0.035919,GC
2,0.0,-0.073871,0.032350,GC
3,0.0,0.118741,0.045421,GC
4,0.0,-0.276701,0.018586,GC


WindowsPath('c:/Users/modre/Documents/masseter/data/processed/erat_outputs.csv')

Sanity checks

In [13]:
assert df_erat_outputs["erat_norm"].between(0, 1).all(), "erat_norm not in [0,1]"
assert np.isfinite(df_erat_outputs["erat_days_pred"]).all(), "Non-finite predictions found"

print(df_erat_outputs[["days_true", "erat_days_pred", "erat_norm"]].describe())
print("Saved:", (paths.processed / "erat_outputs.csv"))

        days_true  erat_days_pred   erat_norm
count  571.000000      571.000000  571.000000
mean     5.490368        5.474443    0.408859
std      4.812757        4.640738    0.314921
min      0.000000       -0.550587    0.000000
25%      2.000000        1.968407    0.170939
50%      5.000000        4.590777    0.348893
75%      7.000000        6.619772    0.486581
max     14.000000       14.185630    1.000000
Saved: c:\Users\modre\Documents\masseter\data\processed\erat_outputs.csv
